In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe modules
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

# Constants for behavior thresholds
HEAD_DEVIATION_THRESHOLD = 10  # Lowered for higher sensitivity
HAND_PROXIMITY_THRESHOLD = 60  # Distance threshold for hand proximity

# Persistent alerts for each type
persistent_head_alert = None
persistent_occlusion_alert = None

def capture_video():
    cap = cv2.VideoCapture(0)  # 0 is the default webcam
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)  # Mirror the frame
        frame = cv2.resize(frame, (640, 480))
        yield frame
    cap.release()

def detect_head_movement(frame, prev_center=None):
    global persistent_head_alert
    head_alert = None
    detection_text = "Head: No deviation"
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.4) as face_detection:
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x, y, w_box, h_box = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)
                current_center = (x + w_box // 2, y + h_box // 2)

                # Check for deviation from initial face position
                if prev_center is not None:
                    deviation = np.linalg.norm(np.array(prev_center) - np.array(current_center))
                    if deviation > HEAD_DEVIATION_THRESHOLD:
                        head_alert = "Abnormal Head Movement!"
                        persistent_head_alert = head_alert
                    else:
                        persistent_head_alert = "Head: No deviation"
                else:
                    persistent_head_alert = "Head: No deviation"
                return current_center
    return prev_center

def detect_face_occlusion(frame, face_landmarks, hand_landmarks):
    global persistent_occlusion_alert
    occlusion_alert = None
    h, w, _ = frame.shape

    # Define bounding boxes around eyes and mouth
    left_eye_points = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in [33, 133, 159, 145, 160, 161, 246, 7]]
    right_eye_points = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in [263, 362, 386, 374, 387, 388, 466, 249]]
    mouth_points = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in [78, 95, 88, 178, 87, 317, 402, 318]]

    # Get bounding box coordinates for each region
    lx_min, ly_min = np.min(left_eye_points, axis=0)
    lx_max, ly_max = np.max(left_eye_points, axis=0)
    rx_min, ry_min = np.min(right_eye_points, axis=0)
    rx_max, ry_max = np.max(right_eye_points, axis=0)
    mx_min, my_min = np.min(mouth_points, axis=0)
    mx_max, my_max = np.max(mouth_points, axis=0)

    # Check for hand overlap with eye and mouth boxes
    occlusion_detected = False
    if hand_landmarks:
        for lm in hand_landmarks.landmark:
            hx, hy = int(lm.x * w), int(lm.y * h)

            # Check for left eye occlusion
            if lx_min <= hx <= lx_max and ly_min <= hy <= ly_max:
                occlusion_alert = "Occlusion detected on eyes!"
                occlusion_detected = True
                break

            # Check for right eye occlusion
            if rx_min <= hx <= rx_max and ry_min <= hy <= ry_max:
                occlusion_alert = "Occlusion detected on eyes!"
                occlusion_detected = True
                break

            # Check for mouth occlusion
            if mx_min <= hx <= mx_max and my_min <= hy <= my_max:
                occlusion_alert = "Occlusion detected on mouth!"
                occlusion_detected = True
                break

    # Update persistent occlusion alert if detected, otherwise reset
    if occlusion_detected:
        persistent_occlusion_alert = occlusion_alert
    else:
        persistent_occlusion_alert = "No occlusion"

    return persistent_occlusion_alert

def detect_hand_movement(frame, face_landmarks):
    occlusion_alert = None
    with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.4) as hands:
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                occlusion_alert = detect_face_occlusion(frame, face_landmarks, hand_landmarks)
    return occlusion_alert

def real_time_cheating_detection():
    head_center = None
    global persistent_head_alert, persistent_occlusion_alert
    persistent_head_alert, persistent_occlusion_alert = "Head: No deviation", "No occlusion"

    for frame in capture_video():
        # Detect head deviation
        head_center = detect_head_movement(frame, head_center)

        # Face detection for hand occlusion checks
        with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
            face_results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if face_results.multi_face_landmarks:
                for face_landmarks in face_results.multi_face_landmarks:
                    occlusion_alert = detect_hand_movement(frame, face_landmarks)

        # Display persistent alerts for head deviation and occlusion
        cv2.putText(frame, persistent_head_alert, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255) if "Abnormal" in persistent_head_alert else (0, 255, 0), 2)
        cv2.putText(frame, persistent_occlusion_alert, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255) if "Occlusion" in persistent_occlusion_alert else (0, 255, 0), 2)

        # Show the frame
        cv2.imshow("Cheating Detection", frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

# Run the real-time cheating detection
real_time_cheating_detection()